In [ ]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import shuffle
from utils import *
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
cars = []
notcars = []

extra_notcars_images = glob.glob('dataset/non-vehicles/Extras/*.png')
for image in extra_notcars_images:
    notcars.append(image)
    
    
gti_notcars_images = glob.glob('dataset/non-vehicles/GTI/*.png')
for image in gti_notcars_images:
    notcars.append(image)
    

gti_far_cars_images = glob.glob('dataset/vehicles/GTI_Far/*.png')
for image in gti_far_cars_images:
    cars.append(image)


gti_left_cars_images = glob.glob('dataset/vehicles/GTI_Left/*.png')
for image in gti_left_cars_images:
    cars.append(image)


gti_middleclose_cars_images = glob.glob('dataset/vehicles/GTI_MiddleClose/*.png')
for image in gti_middleclose_cars_images:
    cars.append(image)


gti_right_cars_images = glob.glob('dataset/vehicles/GTI_Right/*.png')
for image in gti_right_cars_images:
    cars.append(image)


kitti_cars_images = glob.glob('dataset/vehicles/KITTI_extracted/*.png')
for image in kitti_cars_images:
    cars.append(image)


sample_size = 2500
shuffle(cars)
shuffle(notcars)
cars = cars[0:sample_size]
notcars = notcars[0:sample_size]
data_info = data_look(cars, notcars)

print('Your function returned a count of',
      data_info["n_cars"], ' cars and',
      data_info["n_notcars"], ' non-cars')
print('of size: ', data_info["image_shape"], ' and data type:',
      data_info["data_type"])

In [ ]:
# Just for fun choose random car / not-car indices and plot example images
car_ind = np.random.randint(0, len(cars))
notcar_ind = np.random.randint(0, len(notcars))

# Read in car / not-car images
car_image = mpimg.imread(cars[car_ind])
notcar_image = mpimg.imread(notcars[notcar_ind])


# Plot the examples
fig = plt.figure(figsize=(6, 4))
visualize(fig, 1, 2, [car_image, notcar_image],
          ['Example Car Image', 'Example Not-car Image'],
          ['gray', 'gray'])

In [ ]:
orient = 9  # HOG orientations
pix_per_cell = 8  # HOG pixels per cell
cell_per_block = 2  # HOG cells per block
spatial_size = (32, 32)

car_feature_image = cv2.cvtColor(car_image, cv2.COLOR_RGB2YCrCb)
car_Y_channel = car_feature_image[:,:,0]
car_Cr_channel = car_feature_image[:,:,1]
car_Cb_channel = car_feature_image[:,:,2]
# Call our function with vis=True to see an image output
car_Y_features, car_Y_hog_image = get_hog_features(car_Y_channel, orient=orient,
                                       pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                       vis=True, feature_vec=True)
car_Cr_features = get_hog_features(car_Cr_channel, orient=orient,
                                       pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                       vis=False, feature_vec=True)
car_Cb_features = get_hog_features(car_Cb_channel, orient=orient,
                                       pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                       vis=False, feature_vec=True)
car_spatial_Y_feature = bin_spatial(car_Y_channel, spatial_size)
car_spatial_Cr_feature = bin_spatial(car_Cr_channel, spatial_size)
car_spatial_Cb_feature = bin_spatial(car_Cb_channel, spatial_size)

notcar_feature_image = cv2.cvtColor(notcar_image, cv2.COLOR_RGB2YCrCb)
notcar_Y_channel = notcar_feature_image[:,:,0]
notcar_Cr_channel = notcar_feature_image[:,:,1]
notcar_Cb_channel = notcar_feature_image[:,:,2]
# Call our function with vis=True to see an image output
notcar_Y_features, notcar_Y_hog_image = get_hog_features(notcar_Y_channel, orient=orient,
                                       pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                       vis=True, feature_vec=True)
notcar_Cr_features = get_hog_features(notcar_Cr_channel, orient=orient,
                                       pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                       vis=False, feature_vec=True)
notcar_Cb_features = get_hog_features(notcar_Cb_channel, orient=orient,
                                       pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                       vis=False, feature_vec=True)
notcar_spatial_Y_feature = bin_spatial(notcar_Y_channel, spatial_size)
notcar_spatial_Cr_feature = bin_spatial(notcar_Cr_channel, spatial_size)
notcar_spatial_Cb_feature = bin_spatial(notcar_Cb_channel, spatial_size)

fig = plt.figure(figsize=(16, 14))
vis_img_list = [car_Y_channel, car_Y_hog_image, notcar_Y_channel, notcar_Y_hog_image,
                car_Y_channel, car_spatial_Y_feature.reshape(spatial_size), notcar_Y_channel, notcar_spatial_Y_feature.reshape(spatial_size),
                car_Cr_channel, car_spatial_Cr_feature.reshape(spatial_size), notcar_Cr_channel, notcar_spatial_Cr_feature.reshape(spatial_size),
                car_Cb_channel, car_spatial_Cb_feature.reshape(spatial_size), notcar_Cb_channel, notcar_spatial_Cb_feature.reshape(spatial_size)]
vis_title_list = ['Car Y channel', 'Car Y HOG', 'Not-Car Y channel', 'Not-Car Y HOG',
                  'Car Y channel', 'Car Y spatial features', 'Not-Car Y channel', 'Not-Car Y spatial features',
                  'Car Cr channel', 'Car Cr spatial features', 'Not-Car Cr channel', 'Not-Car Cr spatial features',
                  'Car Cb channel', 'Car Cb spatial features', 'Not-Car Cb channel', 'Not-Car Cb spatial features']
cmap_list = ['gray', 'hot', 'gray', 'hot',
             'gray', 'gray', 'gray', 'gray',
             'gray', 'gray', 'gray', 'gray',
             'gray', 'gray', 'gray', 'gray']
visualize(fig, 4, 4, vis_img_list, vis_title_list, cmap_list)

In [ ]:
import pickle

svm_model = pickle.load(open("svm_model.p", "rb"))
svc = svm_model["svc"]
X_scaler = svm_model["X_scaler"]

In [ ]:
import time

color_space = 'YCrCb'  # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 8  # HOG orientations
pix_per_cell = 8  # HOG pixels per cell
cell_per_block = 2  # HOG cells per block
hog_channel = "ALL"  # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32)  # Spatial binning dimensions
hist_bins = 32  # Number of histogram bins
spatial_feat = True  # Spatial features on or off
hist_feat = True  # Histogram features on or off
hog_feat = True  # HOG features on or off

example_images = glob.glob('test_images/*.jpg')
y_start_stop = [340, 650]  # Min and max in y to search in slide_window()
overlap = 0.5

test_images_list = []
test_title_list = []
cmap_list = []

for img_src in example_images:
    t1 = time.time()
    
    img = mpimg.imread(img_src)
    draw_img = np.copy(img)
    img = img.astype(np.float32) / 255
    print(np.min(img), np.max(img))
    
    windows = slide_window(img, x_start_stop=[None, None], y_start_stop=y_start_stop,
                           xy_window=(128, 128), xy_overlap=(overlap, overlap))
    
    hot_windows = search_windows(img, windows, svc, X_scaler, color_space=color_space,
                             spatial_size=spatial_size, hist_bins=hist_bins,
                             orient=orient, pix_per_cell=pix_per_cell,
                             cell_per_block=cell_per_block,
                             hog_channel=hog_channel, spatial_feat=spatial_feat,
                             hist_feat=hist_feat, hog_feat=hog_feat)
    
    window_img = draw_boxes(draw_img, hot_windows, color=(0, 0, 255), thick=6)
    test_images_list.append(window_img)
    test_title_list.append('')
    cmap_list.append('gray')
    print(time.time() - t1, 'seconds to process one image searching', len(windows), 'windows')
    
fig = plt.figure(figsize=(12, 18), dpi=300)
visualize(fig, len(example_images), 2, test_images_list, test_title_list, cmap_list)

In [ ]:
out_images = []
out_maps = []
out_titles = []
out_boxes = []
out_cmap = []

ystart = 340
ystop = 650
scale = 2

for img_src in example_images:
    img_boxes = []
    t = time.time()
    count = 0
    img = mpimg.imread(img_src)
    draw_img = np.copy(img)
    
    heatmap = np.zeros_like(img[:,:,0])
    img = img.astype(np.float32) / 255
    
    img_tosearch = img[ystart:ystop, :, :]
    ctrans_tosearch = convert_color(img_tosearch)
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), (np.int(imshape[0]/scale))))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]
    
    nxblocks = (ch1.shape[1] // pix_per_cell) - 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - 1
    nfeat_per_block = orient * cell_per_block**2
    
    window = 64
    nblocks_per_window = (window // pix_per_cell) - 1
    cells_per_step = 2
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            count += 1
            ypos = yb * cells_per_step
            xpos = xb * cells_per_step
            
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
            
            xleft = xpos * pix_per_cell
            ytop = ypos * pix_per_cell
            
            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64, 64))
            
            # Get features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)
            
            features = np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1)
            
            # Scale extracted features to be fed to classifier
            test_features = X_scaler.transform(features)
            # Predict using your classifier
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft * scale)
                ytop_draw = np.int(ytop * scale)
                win_draw = np.int(window * scale)
                cv2.rectangle(draw_img, (xbox_left, ytop_draw+ystart),
                              (xbox_left+win_draw, ytop_draw+win_draw+ystart),
                              (0, 0, 255), 6)
                img_boxes.append(((xbox_left, ytop_draw+ystart),
                                  (xbox_left+win_draw, ytop_draw+win_draw+ystart)))
                heatmap[ytop_draw+ystart:ytop_draw+win_draw+ystart,
                        xbox_left:xbox_left+win_draw] += 1
                
    print(time.time() - t, 'seconds to run, total windows = ', count)
    
    out_images.append(draw_img)
    out_cmap.append('gray')
    out_titles.append(img_src[-9:])
    out_titles.append(img_src[-9:])
    out_images.append(heatmap)
    out_cmap.append('hot')
    out_boxes.append(img_boxes)
        

fig = plt.figure(figsize=(14, 40), dpi=300)
visualize(fig, 2*len(example_images), 2, out_images, out_titles, out_cmap)


In [ ]:
def find_cars(img, scale):
    draw_img = np.copy(img)
    
    heatmap = np.zeros_like(img[:,:,0])
    img = img.astype(np.float32) / 255
    
    img_tosearch = img[ystart:ystop, :, :]
    ctrans_tosearch = convert_color(img_tosearch)
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), (np.int(imshape[0]/scale))))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]
    
    nxblocks = (ch1.shape[1] // pix_per_cell) - 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - 1
    nfeat_per_block = orient * cell_per_block**2
    
    window = 64
    nblocks_per_window = (window // pix_per_cell) - 1
    cells_per_step = 2
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb * cells_per_step
            xpos = xb * cells_per_step
            
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
            
            xleft = xpos * pix_per_cell
            ytop = ypos * pix_per_cell
            
            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64, 64))
            
            # Get features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)
            
            features = np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1)
            
            # Scale extracted features to be fed to classifier
            test_features = X_scaler.transform(features)
            # Predict using your classifier
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft * scale)
                ytop_draw = np.int(ytop * scale)
                win_draw = np.int(window * scale)
                cv2.rectangle(draw_img, (xbox_left, ytop_draw+ystart),
                              (xbox_left+win_draw, ytop_draw+win_draw+ystart),
                              (0, 0, 255), 6)
                img_boxes.append(((xbox_left, ytop_draw+ystart),
                                  (xbox_left+win_draw, ytop_draw+win_draw+ystart)))
                heatmap[ytop_draw+ystart:ytop_draw+win_draw+ystart,
                        xbox_left:xbox_left+win_draw] += 1
                
    return draw_img, heatmap

In [ ]:
from scipy.ndimage.measurements import label

out_images = []
out_maps = []
out_titles = []
out_boxes = []
out_cmap = []

ystart = 340
ystop = 650
scale = 2

for img_src in example_images:
    img = mpimg.imread(img_src)
    out_img, heat_map = find_cars(img, scale)
    
    heat = apply_threshold(heat_map, 1)
    labels = label(heat)
    
    draw_img = draw_labeled_bboxes(np.copy(img), labels)
    
    out_images.append(draw_img)
    out_titles.append('')
    out_cmap.append('gray')
    out_images.append(heat_map)
    out_titles.append('')
    out_cmap.append('hot')
    
    
fig = plt.figure(figsize=(14, 40), dpi=300)
visualize(fig, 2*len(example_images), 2, out_images, out_titles, out_cmap)

In [ ]:
def process_image(img):
    out_img, heat_map = find_cars(img, scale)
    
    heat = apply_threshold(heat_map, 2)
    labels = label(heat)
    
    draw_img = draw_labeled_bboxes(np.copy(img), labels)
    return draw_img

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

video_file = 'project_video.mp4'
track_output = 'track_' + video_file
clip = VideoFileClip(video_file)
track_clip = clip.fl_image(process_image)
track_clip.write_videofile(track_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(track_output))

In [ ]:
class Vehicle():
    def __init__(self):
        self.detected = False # was the Vehicle detected in the last iteration?
        self.n_detections = 0 # Number of times this vehicle has been?
        self.n_nondetections = 0 # Number of consecutive times this car hs no been detected
        self.xpixels = None # Pixel x values of last detection
        self.ypixels = None # pixel y values of last detection
        self.recent_xfitted = [] # x position of the last n fits
        self.bestx = None # average x position of the last n fits
        self.recent_hfitted = [] # y position of the last n fits
        self.besty = None # average y position of the last n fits
        self.recent_wfitted = [] # width of the last n fits of the bounding box
        self.bestw = None # average width of the last n fits
        self.recent_hfitted = [] # height of the last n fits of the bounding box
        self.besth = None # average height of the last n fits